In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# import sequence from tensorflow
from tensorflow.keras.utils import Sequence

In [2]:
def load_images_from_folder(folder):
    images = []
    labels = []
    label = 0
    for person_folder in os.listdir(folder):
        person_path = os.path.join(folder, person_folder)
        if os.path.isdir(person_path):
            for image_name in os.listdir(person_path):
                img_path = os.path.join(person_path, image_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (160, 160))
                images.append(img)
                labels.append(label)
            label += 1
    return np.array(images), np.array(labels)

def make_pairs(images, labels):
    pair_images = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    class_idx = [np.where(labels == i)[0] for i in range(num_classes)]

    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]
        
        # Make a positive pair
        idx2 = np.random.choice(class_idx[label])
        pos_image = images[idx2]
        
        pair_images.append([current_image, pos_image])
        pair_labels.append([1])
        
        # Make a negative pair
        neg_label = np.random.choice(np.delete(np.arange(num_classes), label))
        idx2 = np.random.choice(class_idx[neg_label])
        neg_image = images[idx2]
        
        pair_images.append([current_image, neg_image])
        pair_labels.append([0])

    return np.array(pair_images), np.array(pair_labels)

In [3]:
def create_siamese_model(input_shape):
    input = layers.Input(input_shape)
    x = layers.Conv2D(64, (10, 10), activation='relu')(input)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (7, 7), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (4, 4), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, (4, 4), activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(4096, activation='sigmoid')(x)
    model = Model(input, x)
    return model

In [4]:
def build_siamese_network(input_shape):
    base_model = create_siamese_model(input_shape)
    
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)
    
    processed_a = base_model(input_a)
    processed_b = base_model(input_b)
    
    distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([processed_a, processed_b])
    outputs = layers.Dense(1, activation='sigmoid')(distance)
    
    model = Model([input_a, input_b], outputs)
    return model

In [5]:
class SiameseDataGenerator(Sequence):
    def __init__(self, image_pairs, labels, batch_size, image_shape):
        self.image_pairs = image_pairs
        self.labels = labels
        self.batch_size = batch_size
        self.image_shape = image_shape
        self.indices = np.arange(len(self.image_pairs))
    
    def __len__(self):
        return len(self.image_pairs) // self.batch_size
    
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_pairs = [self.image_pairs[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]
        
        x1 = np.zeros((self.batch_size,) + self.image_shape, dtype="float32")
        x2 = np.zeros((self.batch_size,) + self.image_shape, dtype="float32")
        y = np.zeros((self.batch_size,), dtype="float32")
        
        for i, (img1_path, img2_path) in enumerate(batch_pairs):
            img1 = preprocess_image(img1_path)
            img2 = preprocess_image(img2_path)
            x1[i] = img1[0]  # Remove the batch dimension
            x2[i] = img2[0]  # Remove the batch dimension
            y[i] = batch_labels[i]  # Ensure batch_labels[i] is a float value
        
        return [x1, x2], y
    
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [6]:
from deepface import DeepFace

In [7]:
def preprocess_image(image_path):
    """
    Preprocess the image to match the input shape required by the model.
    """
    aligned_face = Deepface.detect_face(img = cv2.imread(image_path), detector_backend = 'mtcnn')
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    img = cv2.resize(img, (160, 160))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def make_pairs_with_paths(image_paths, labels):
    pair_paths = []
    pair_labels = []
    unique_labels = np.unique(labels)
    label_to_indices = {label: np.where(labels == label)[0] for label in unique_labels}

    for idx1 in range(len(image_paths)):
        current_image_path = image_paths[idx1]
        label = labels[idx1]
        
        # Make a positive pair
        if len(label_to_indices[label]) > 0:
            idx2 = np.random.choice(label_to_indices[label])
            pos_image_path = image_paths[idx2]
            pair_paths.append([current_image_path, pos_image_path])
            pair_labels.append(1)  # Ensure this is a numeric value
        
        # Make a negative pair
        neg_label_candidates = np.delete(unique_labels, np.where(unique_labels == label))
        neg_label = np.random.choice(neg_label_candidates)
        
        if len(label_to_indices[neg_label]) > 0:
            idx2 = np.random.choice(label_to_indices[neg_label])
            neg_image_path = image_paths[idx2]
            pair_paths.append([current_image_path, neg_image_path])
            pair_labels.append(0)  # Ensure this is a numeric value

    return pair_paths, np.array(pair_labels)

# Load paths
image_paths = []
labels = []
for label in os.listdir('Extracted Faces'):
    for image_file in os.listdir(os.path.join('Extracted Faces', label)):
        image_paths.append(os.path.join('Extracted Faces', label, image_file))
        labels.append(int(label))

# Generate pairs with paths
pair_paths, pair_labels = make_pairs_with_paths(image_paths, labels)

# Split data
train_paths, test_paths, train_labels, test_labels = train_test_split(pair_paths, pair_labels, test_size=0.2, random_state=42)

# Create data generators
train_generator = SiameseDataGenerator(train_paths, train_labels, batch_size=16, image_shape=(160, 160, 3))
test_generator = SiameseDataGenerator(test_paths, test_labels, batch_size=16, image_shape=(160, 160, 3))

In [9]:
siamese_network.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 4096)      │ 152,206,6… │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 4096)      │          0 │ functional[0][0], │
│                     │                   │            │ functional[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      4,097 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 152,210,753 (580.64 MB)

 Trainable params: 152,210,753 (580.64 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
input_shape = (160, 160, 3)
siamese_network = build_siamese_network(input_shape)
siamese_network.compile(loss="binary_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])

# Train the model using the generator
history = siamese_network.fit(train_generator, validation_data=test_generator, epochs=20)

NameError: name 'deepface' is not defined

In [9]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# Evaluate on test data
test_loss, test_acc = siamese_network.evaluate([pair_images_test[:, 0], pair_images_test[:, 1]], pair_labels_test)

print(f"Test accuracy: {test_acc * 100:.2f}%")

NameError: name 'history' is not defined

In [50]:
image_file1 = './sayar2.jpg'
image_file2 = './sayar3.jpg'

img1 = preprocess_image(image_file1)
img2 = preprocess_image(image_file2)

x1 = np.zeros((1,) + img1[0].shape, dtype="float32")
x2 = np.zeros((1,) + img2[0].shape, dtype="float32")
x1[0] = img1[0]
x2[0] = img2[0]

y_pred = siamese_network.predict([x1, x2])
print(y_pred)
if y_pred[0][0] < 0.51:
    print("Not Verified")
else:
    print("Verified")

[[0.5113201]]
Verified


In [8]:
siamese_network.save_weights('siamese_network.h5')

In [ ]:
siamese_network.load_weights('siamese_network.h5')